<a href="https://colab.research.google.com/github/sanketghosh19/Crypto_Price_Prediction/blob/main/Crypto_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries in Colab
!pip install pandas scikit-learn

# Import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
# Load dataset
from google.colab import files
uploaded = files.upload()

# Load Excel file
file_name = list(uploaded.keys())[0]
data = pd.ExcelFile(file_name)

# Combine all sheets into one DataFrame
all_data = {}
for sheet_name in data.sheet_names:
    try:
        sheet_data = data.parse(sheet_name)
        all_data[sheet_name] = sheet_data
    except Exception as e:
        print(f"Error reading sheet {sheet_name}: {e}")

combined_data = pd.concat(all_data.values(), keys=all_data.keys(), names=["Coin", "Index"])

Saving Output_database (2).xlsx to Output_database (2).xlsx


<ipython-input-2-80ca3c5ed345>:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat(all_data.values(), keys=all_data.keys(), names=["Coin", "Index"])
<ipython-input-2-80ca3c5ed345>:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat(all_data.values(), keys=all_data.keys(), names=["Coin", "Index"])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# Function to extract features with filtering conditions
def extract_features(data):
    prices = data[['date', 'price']].dropna().sort_values('date')
    if len(prices) < 2:
        return None

    prices['date'] = pd.to_datetime(prices['date'])
    first_date = prices['date'].iloc[0]
    last_date = prices['date'].iloc[-1]
    age_in_days = (last_date - first_date).days
    age_in_months = round(age_in_days / 30.44, 2)

    # Calculate percentage price changes
    def calculate_price_change(days):
        if len(prices) >= days:
            start_price = prices['price'].iloc[-days]
            end_price = prices['price'].iloc[-1]
            return ((end_price - start_price) / start_price) * 100 if start_price != 0 else 0
        return None

    price_changes = {
        'price_change_24h': calculate_price_change(2),
        'price_change_7d': calculate_price_change(7),
        'price_change_14d': calculate_price_change(14),
        'price_change_30d': calculate_price_change(30)
    }

    for key in price_changes:
        if price_changes[key] is None:
            price_changes[key] = 0  # or any other suitable default value


    # Filter out coins based on conditions
    if (price_changes['price_change_24h'] < 0 and price_changes['price_change_7d'] < 0) or (price_changes['price_change_30d'] < -50):
        return None

    features = {
        'mean_price': np.mean(prices['price']),
        'std_price': np.std(prices['price']),
        'min_price': np.min(prices['price']),
        'max_price': np.max(prices['price']),
        'volatility': np.std(prices['price']) / np.mean(prices['price']) if np.mean(prices['price']) != 0 else 0,
        'price_change': (prices['price'].iloc[-1] - prices['price'].iloc[0]) / prices['price'].iloc[0] if prices['price'].iloc[0] != 0 else 0,
        'token': data['token'].iloc[0] if 'token' in data.columns else None,
        'contract_address': data['contract_address'].iloc[0] if 'contract_address' in data.columns else None,
        'market_cap': data['market_cap'].dropna().iloc[-1] if 'market_cap' in data.columns else None,
        'age_in_months': age_in_months,
        'Chain': data['platform'].iloc[0] if 'platform' in data.columns else None,
        'Trading Volume': data['volume'].dropna().iloc[-1] if 'volume' in data.columns else None,
        'twitter_followers': data['twitter_followers'].iloc[0] if 'twitter_followers' in data.columns else None,
        'price': prices['price'].iloc[-1]
    }

    features.update(price_changes)
    return features

# Extract features for each coin and apply filters
feature_data = []
for coin, coin_data in combined_data.groupby(level=0):
    features = extract_features(coin_data)
    if features:
        features['Coin'] = coin
        feature_data.append(features)

# Convert to DataFrame
features_df = pd.DataFrame(feature_data)

# Format columns for display
features_df['market_cap'] = features_df['market_cap'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)
features_df['Trading Volume'] = features_df['Trading Volume'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)

# Convert columns to numeric for clustering
features_df['Trading Volume'] = features_df['Trading Volume'].str.replace(',', '').astype(float)
features_df['market_cap'] = features_df['market_cap'].str.replace(',', '').astype(float)

# Normalize numerical columns
features_normalized = features_df[['mean_price', 'std_price', 'volatility', 'price_change']].apply(
    lambda x: (x - x.mean()) / x.std()
)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
features_df['Cluster'] = kmeans.fit_predict(features_normalized)

# Map clusters to probability groups based on price change
cluster_mapping = (
    features_df.groupby('Cluster')['price_change']
    .mean()
    .sort_values(ascending=False)
    .index
)
features_df['Probability_Group'] = features_df['Cluster'].map(
    {cluster_mapping[0]: '90% Uptrend',
     cluster_mapping[1]: '80% Uptrend',
     cluster_mapping[2]: '70% Uptrend'}
)

# Filter based on trading volume and market cap
filtered_features = features_df[
    (features_df['Trading Volume'] >= 50000) & (features_df['market_cap'] >= 1_000_000)
]

# Select exactly 20 coins per probability group
final_clusters = (
    filtered_features.groupby('Probability_Group', group_keys=False)
    .apply(lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20))
)

# Reformat columns for display
final_clusters['market_cap'] = final_clusters['market_cap'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)
final_clusters['Trading Volume'] = final_clusters['Trading Volume'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)

# Display final clusters
print(final_clusters)

       mean_price     std_price     min_price     max_price  volatility  \
113  2.824346e+02  4.861554e+01  1.795970e+02  3.815710e+02    0.172130   
148  8.511364e-08  2.359672e-08  5.000000e-08  2.100000e-07    0.277238   
18   1.826719e-02  1.025091e-02  8.491600e-03  3.800430e-02    0.561165   
7    3.192554e-02  8.962780e-03  1.506770e-02  4.961350e-02    0.280740   
156  8.516636e-05  3.057200e-05  4.085000e-05  1.596000e-04    0.358968   
122  4.550650e-02  6.284300e-03  3.922220e-02  5.179080e-02    0.138097   
130  6.181818e-08  7.767276e-09  5.000000e-08  7.000000e-08    0.125647   
55   5.196545e-03  1.497496e-03  2.588360e-03  8.438080e-03    0.288171   
88   6.409087e-03  1.937589e-03  2.750240e-03  1.041550e-02    0.302319   
175  4.228770e-03  1.424566e-03  2.180370e-03  8.279090e-03    0.336875   
19   1.750000e-08  4.826536e-09  1.000000e-08  3.000000e-08    0.275802   
150  1.181818e-08  3.856946e-09  1.000000e-08  2.000000e-08    0.326357   
152  1.789773e-02  3.5510

<ipython-input-3-bc6255485c63>:110: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20))


In [ ]:
# Save clusters to CSV
final_clusters.to_csv("final_clustered_coins.csv", index=False)

# Download the file (if using Colab)
from google.colab import files
files.download("final_clustered_coins.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title NEW
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
# For incremental learning, you might consider using MiniBatchKMeans or an online regression model (e.g., SGDRegressor)

# ============
# FUNCTIONS
# ============

def extract_features(data):
    # Ensure date is datetime and sort by date
    prices = data[['date', 'price']].dropna().sort_values('date')
    if len(prices) < 2:
        return None

    prices['date'] = pd.to_datetime(prices['date'])
    first_date = prices['date'].iloc[0]
    last_date = prices['date'].iloc[-1]
    age_in_days = (last_date - first_date).days
    age_in_months = round(age_in_days / 30.44, 2)

    # Function to calculate percentage change over the last 'days' days
    def calculate_price_change(days):
        if len(prices) >= days:
            start_price = prices['price'].iloc[-days]
            end_price = prices['price'].iloc[-1]
            return ((end_price - start_price) / start_price) * 100 if start_price != 0 else 0
        return None

    price_changes = {
        'price_change_24h': calculate_price_change(2),
        'price_change_7d': calculate_price_change(7),
        'price_change_14d': calculate_price_change(14),
        'price_change_30d': calculate_price_change(30)
    }

    # Replace missing changes with default values (here, 0)
    for key in price_changes:
        if price_changes[key] is None:
            price_changes[key] = 0

    # Filter out coins based on conditions
    if (price_changes['price_change_24h'] < 0 and price_changes['price_change_7d'] < 0) or (price_changes['price_change_30d'] < -50):
        return None

    features = {
        'mean_price': np.mean(prices['price']),
        'std_price': np.std(prices['price']),
        'min_price': np.min(prices['price']),
        'max_price': np.max(prices['price']),
        'volatility': np.std(prices['price']) / np.mean(prices['price']) if np.mean(prices['price']) != 0 else 0,
        'price_change': (prices['price'].iloc[-1] - prices['price'].iloc[0]) / prices['price'].iloc[0] if prices['price'].iloc[0] != 0 else 0,
        'token': data['token'].iloc[0] if 'token' in data.columns else None,
        'contract_address': data['contract_address'].iloc[0] if 'contract_address' in data.columns else None,
        'market_cap': data['market_cap'].dropna().iloc[-1] if 'market_cap' in data.columns else None,
        'age_in_months': age_in_months,
        'Chain': data['platform'].iloc[0] if 'platform' in data.columns else None,
        'Trading Volume': data['volume'].dropna().iloc[-1] if 'volume' in data.columns else None,
        'twitter_followers': data['twitter_followers'].iloc[0] if 'twitter_followers' in data.columns else None,
        'price': prices['price'].iloc[-1]
    }

    features.update(price_changes)
    return features

def record_predictions(prediction_df, record_file='predictions_log.csv'):
    """
    Record the prediction data (with a timestamp) to a CSV log.
    """
    prediction_df = prediction_df.copy()
    prediction_df['prediction_date'] = pd.Timestamp.now()
    # Append to file if exists; otherwise, create new file with header.
    if os.path.exists(record_file):
        prediction_df.to_csv(record_file, mode='a', header=False, index=False)
    else:
        prediction_df.to_csv(record_file, index=False)
    print(f"Predictions recorded to {record_file}")

def evaluate_predictions(n_days=3, record_file='predictions_log.csv', updated_data=None):
    """
    Evaluate predictions made more than n_days ago by comparing the predicted price with the latest actual price.
    `updated_data` is assumed to be the latest combined_data DataFrame with up-to-date prices.
    """
    if not os.path.exists(record_file):
        print("No prediction log file found for evaluation.")
        return

    predictions_log = pd.read_csv(record_file)
    predictions_log['prediction_date'] = pd.to_datetime(predictions_log['prediction_date'])
    cutoff_date = pd.Timestamp.now() - timedelta(days=n_days)

    # Filter predictions older than n_days
    old_predictions = predictions_log[predictions_log['prediction_date'] <= cutoff_date]
    if old_predictions.empty:
        print(f"No predictions older than {n_days} days to evaluate.")
        return

    errors = []
    error_details = []

    # Loop over each old prediction and compute the error between the predicted price and current actual price.
    for idx, row in old_predictions.iterrows():
        coin = row['Coin']
        predicted_price = row['price']
        try:
            # Assumes that `updated_data` has the latest price info, indexed (or grouped) by coin.
            coin_data = updated_data.loc[coin]
            if isinstance(coin_data, pd.DataFrame):
                # If multiple rows exist, use the latest non-NA price
                actual_price = coin_data['price'].dropna().iloc[-1]
            else:
                actual_price = coin_data['price']
        except Exception as e:
            print(f"Could not retrieve data for coin {coin}: {e}")
            continue

        error = actual_price - predicted_price
        errors.append(abs(error))
        error_details.append({
            'Coin': coin,
            'Predicted_Price': predicted_price,
            'Actual_Price': actual_price,
            'Error': error,
            'Prediction_Date': row['prediction_date']
        })

    if errors:
        mae = np.mean(errors)
        print(f"Mean Absolute Error over predictions older than {n_days} days: {mae:,.4f}")
        # Here, you might want to save error_details to a file or database for further analysis.
        errors_df = pd.DataFrame(error_details)
        print(errors_df)
    else:
        print("No valid predictions were evaluated.")

# ============
# MAIN PROCESSING PIPELINE
# ============

# Assume combined_data is a DataFrame containing historical and current data for each coin.
# For example, it might have a MultiIndex (coin, timestamp) or similar structure.
# combined_data = pd.read_csv('your_combined_data.csv', index_col=0)  # This is just an example.

# Extract features for each coin and apply filtering conditions
feature_data = []
# Assuming combined_data is grouped by coin identifier (e.g., coin name or symbol)
for coin, coin_data in combined_data.groupby(level=0):
    features = extract_features(coin_data)
    if features:
        features['Coin'] = coin
        feature_data.append(features)

# Convert list of feature dictionaries to a DataFrame
features_df = pd.DataFrame(feature_data)

# Format market_cap and Trading Volume columns for display
features_df['market_cap'] = features_df['market_cap'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)
features_df['Trading Volume'] = features_df['Trading Volume'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)

# Convert formatted strings back to numeric for clustering
features_df['Trading Volume'] = features_df['Trading Volume'].str.replace(',', '').astype(float)
features_df['market_cap'] = features_df['market_cap'].str.replace(',', '').astype(float)

# Normalize selected numerical columns for clustering
features_normalized = features_df[['mean_price', 'std_price', 'volatility', 'price_change']].apply(
    lambda x: (x - x.mean()) / x.std()
)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
features_df['Cluster'] = kmeans.fit_predict(features_normalized)

# Map clusters to probability groups based on average price_change
cluster_mapping = (
    features_df.groupby('Cluster')['price_change']
    .mean()
    .sort_values(ascending=False)
    .index
)
features_df['Probability_Group'] = features_df['Cluster'].map(
    {cluster_mapping[0]: '90% Uptrend',
     cluster_mapping[1]: '80% Uptrend',
     cluster_mapping[2]: '70% Uptrend'}
)

# Filter based on trading volume and market cap thresholds
filtered_features = features_df[
    (features_df['Trading Volume'] >= 50000) & (features_df['market_cap'] >= 1_000_000)
]

# Select exactly 20 coins per probability group (or as many as available if fewer than 20)
final_clusters = (
    filtered_features.groupby('Probability_Group', group_keys=False)
    .apply(lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20))
)

# Reformat columns for display
final_clusters['market_cap'] = final_clusters['market_cap'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)
final_clusters['Trading Volume'] = final_clusters['Trading Volume'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)

# Display final clusters (predictions)
print("Final Clusters (Predictions):")
print(final_clusters)

# ========
# RECORD THE PREDICTIONS
# ========

# Save/log the prediction results (including predicted price and probability group) for future feedback evaluation.
record_predictions(final_clusters, record_file='predictions_log.csv')

# ============
# FEEDBACK LOOP (To be run after n days)
# ============

# At a later time (after n days), you would run an evaluation routine.
# For example, you might schedule the following function (e.g., via a cron job or scheduled task):
# Make sure that combined_data has been updated with the latest market data before evaluation.
evaluate_predictions(n_days=3, record_file='predictions_log.csv', updated_data=combined_data)

# ============
# SUGGESTIONS FOR FURTHER IMPROVEMENTS:
# ============
# 1. **Automated Retraining:** Instead of only logging and evaluating, consider re-computing the features
#    and retraining (or re-clustering) periodically so that your model captures the latest market dynamics.
#
# 2. **Use Incremental/Online Learning:** If you switch to a supervised framework for predicting future price changes,
#    consider models that support incremental learning (e.g., SGDRegressor, MiniBatchKMeans, or online neural networks).
#
# 3. **Feature Expansion:** Consider adding additional features (e.g., technical indicators, social sentiment scores)
#    to improve predictive power.
#
# 4. **Error-Driven Adjustments:** Use the computed error metrics (e.g., MAE, RMSE) to adjust thresholds,
#    re-weight features, or even update the mapping from clusters to probability groups.
#
# 5. **Data Persistence:** Instead of CSV logs, you might use a database (e.g., SQLite, PostgreSQL) to store historical
#    predictions and evaluation metrics for more robust long-term tracking and model updates.
#
# 6. **Model Monitoring and Alerts:** Implement monitoring to alert you when prediction errors exceed a threshold,
#    which could indicate that market conditions have changed and the model needs retraining.


Final Clusters (Predictions):
       mean_price     std_price     min_price     max_price  volatility  \
113  2.824346e+02  4.861554e+01  1.795970e+02  3.815710e+02    0.172130   
148  8.511364e-08  2.359672e-08  5.000000e-08  2.100000e-07    0.277238   
18   1.826719e-02  1.025091e-02  8.491600e-03  3.800430e-02    0.561165   
7    3.192554e-02  8.962780e-03  1.506770e-02  4.961350e-02    0.280740   
156  8.516636e-05  3.057200e-05  4.085000e-05  1.596000e-04    0.358968   
122  4.550650e-02  6.284300e-03  3.922220e-02  5.179080e-02    0.138097   
130  6.181818e-08  7.767276e-09  5.000000e-08  7.000000e-08    0.125647   
55   5.196545e-03  1.497496e-03  2.588360e-03  8.438080e-03    0.288171   
88   6.409087e-03  1.937589e-03  2.750240e-03  1.041550e-02    0.302319   
175  4.228770e-03  1.424566e-03  2.180370e-03  8.279090e-03    0.336875   
19   1.750000e-08  4.826536e-09  1.000000e-08  3.000000e-08    0.275802   
150  1.181818e-08  3.856946e-09  1.000000e-08  2.000000e-08    0.32635

<ipython-input-7-73a423b23e7e>:197: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20))


In [ ]:
#@title NEW_MOD
import pandas as pd
import numpy as np
import os
import time  # for sleep between retries
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
# For incremental learning, you might consider using MiniBatchKMeans or an online regression model (e.g., SGDRegressor)

# ============
# FUNCTIONS
# ============

def extract_features(data):
    # Ensure date is datetime and sort by date
    prices = data[['date', 'price']].dropna().sort_values('date')
    if len(prices) < 2:
        return None

    prices['date'] = pd.to_datetime(prices['date'])
    first_date = prices['date'].iloc[0]
    last_date = prices['date'].iloc[-1]
    age_in_days = (last_date - first_date).days
    age_in_months = round(age_in_days / 30.44, 2)

    # Function to calculate percentage change over the last 'days' days
    def calculate_price_change(days):
        if len(prices) >= days:
            start_price = prices['price'].iloc[-days]
            end_price = prices['price'].iloc[-1]
            return ((end_price - start_price) / start_price) * 100 if start_price != 0 else 0
        return None

    price_changes = {
        'price_change_24h': calculate_price_change(2),
        'price_change_7d': calculate_price_change(7),
        'price_change_14d': calculate_price_change(14),
        'price_change_30d': calculate_price_change(30)
    }

    # Replace missing changes with default values (here, 0)
    for key in price_changes:
        if price_changes[key] is None:
            price_changes[key] = 0

    # Filter out coins based on conditions
    if (price_changes['price_change_24h'] < 0 and price_changes['price_change_7d'] < 0) or (price_changes['price_change_30d'] < -50):
        return None

    features = {
        'mean_price': np.mean(prices['price']),
        'std_price': np.std(prices['price']),
        'min_price': np.min(prices['price']),
        'max_price': np.max(prices['price']),
        'volatility': np.std(prices['price']) / np.mean(prices['price']) if np.mean(prices['price']) != 0 else 0,
        'price_change': (prices['price'].iloc[-1] - prices['price'].iloc[0]) / prices['price'].iloc[0] if prices['price'].iloc[0] != 0 else 0,
        'token': data['token'].iloc[0] if 'token' in data.columns else None,
        'contract_address': data['contract_address'].iloc[0] if 'contract_address' in data.columns else None,
        'market_cap': data['market_cap'].dropna().iloc[-1] if 'market_cap' in data.columns else None,
        'age_in_months': age_in_months,
        'Chain': data['platform'].iloc[0] if 'platform' in data.columns else None,
        'Trading Volume': data['volume'].dropna().iloc[-1] if 'volume' in data.columns else None,
        'twitter_followers': data['twitter_followers'].iloc[0] if 'twitter_followers' in data.columns else None,
        'price': prices['price'].iloc[-1]
    }

    features.update(price_changes)
    return features

def record_predictions(prediction_df, record_file='predictions_log.csv'):
    """
    Record the prediction data (with a timestamp) to a CSV log.
    """
    prediction_df = prediction_df.copy()
    prediction_df['prediction_date'] = pd.Timestamp.now()
    # Append to file if exists; otherwise, create new file with header.
    if os.path.exists(record_file):
        prediction_df.to_csv(record_file, mode='a', header=False, index=False)
    else:
        prediction_df.to_csv(record_file, index=False)
    print(f"Predictions recorded to {record_file}")

def evaluate_predictions(n_days=3, record_file='predictions_log.csv', updated_data=None):
    """
    Evaluate predictions made more than n_days ago by comparing the predicted price with the latest actual price.
    `updated_data` is assumed to be the latest combined_data DataFrame with up-to-date prices.
    """
    if not os.path.exists(record_file):
        print("No prediction log file found for evaluation.")
        return

    predictions_log = pd.read_csv(record_file)
    predictions_log['prediction_date'] = pd.to_datetime(predictions_log['prediction_date'])
    cutoff_date = pd.Timestamp.now() - timedelta(days=n_days)

    # Filter predictions older than n_days
    old_predictions = predictions_log[predictions_log['prediction_date'] <= cutoff_date]
    if old_predictions.empty:
        print(f"No predictions older than {n_days} days to evaluate.")
        return

    errors = []
    error_details = []

    # Loop over each old prediction and compute the error between the predicted price and current actual price.
    for idx, row in old_predictions.iterrows():
        coin = row['Coin']
        predicted_price = row['price']
        try:
            # Assumes that `updated_data` has the latest price info, indexed (or grouped) by coin.
            coin_data = updated_data.loc[coin]
            if isinstance(coin_data, pd.DataFrame):
                # If multiple rows exist, use the latest non-NA price
                actual_price = coin_data['price'].dropna().iloc[-1]
            else:
                actual_price = coin_data['price']
        except Exception as e:
            print(f"Could not retrieve data for coin {coin}: {e}")
            continue

        error = actual_price - predicted_price
        errors.append(abs(error))
        error_details.append({
            'Coin': coin,
            'Predicted_Price': predicted_price,
            'Actual_Price': actual_price,
            'Error': error,
            'Prediction_Date': row['prediction_date']
        })

    if errors:
        mae = np.mean(errors)
        print(f"Mean Absolute Error over predictions older than {n_days} days: {mae:,.4f}")
        # Optionally, save error_details to a file or database for further analysis.
        errors_df = pd.DataFrame(error_details)
        print(errors_df)
    else:
        print("No valid predictions were evaluated.")

def retrain_model(combined_data, record_file='predictions_log.csv'):
    """
    Re-compute features, re-cluster coins, and record new predictions.
    This function is wrapped in a try/except block so that errors are caught and logged,
    allowing the retraining process to be retried later.
    """
    try:
        print("Starting model retraining...")
        feature_data = []
        # Assuming combined_data is a DataFrame with a MultiIndex where level=0 is the coin identifier.
        for coin, coin_data in combined_data.groupby(level=0):
            features = extract_features(coin_data)
            if features:
                features['Coin'] = coin
                feature_data.append(features)
        if not feature_data:
            print("No valid features extracted for retraining.")
            return

        features_df = pd.DataFrame(feature_data)

        # Convert 'market_cap' and 'Trading Volume' to numeric (if needed)
        for col in ['market_cap', 'Trading Volume']:
            if col in features_df.columns:
                features_df[col] = pd.to_numeric(features_df[col], errors='coerce')

        # Normalize selected numerical columns for clustering
        norm_columns = ['mean_price', 'std_price', 'volatility', 'price_change']
        features_normalized = features_df[norm_columns].apply(lambda x: (x - x.mean()) / x.std())

        # Apply K-Means clustering
        kmeans = KMeans(n_clusters=3, random_state=42)
        features_df['Cluster'] = kmeans.fit_predict(features_normalized)

        # Map clusters to probability groups based on average price_change
        cluster_mapping = features_df.groupby('Cluster')['price_change'].mean().sort_values(ascending=False).index
        # Create a mapping assuming three clusters exist
        mapping_dict = {}
        if len(cluster_mapping) >= 3:
            mapping_dict = {cluster_mapping[0]: '90% Uptrend',
                            cluster_mapping[1]: '80% Uptrend',
                            cluster_mapping[2]: '70% Uptrend'}
        features_df['Probability_Group'] = features_df['Cluster'].map(mapping_dict)

        # Filter based on trading volume and market cap thresholds
        filtered_features = features_df[
            (features_df['Trading Volume'] >= 50000) & (features_df['market_cap'] >= 1_000_000)
        ]

        # Select exactly 20 coins per probability group (or as many as available if fewer than 20)
        final_clusters = filtered_features.groupby('Probability_Group', group_keys=False).apply(
            lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20)
        )

        # Optionally, reformat market_cap and Trading Volume for display purposes
        for col in ['market_cap', 'Trading Volume']:
            final_clusters[col] = final_clusters[col].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)

        print("Retrained Clusters (Predictions):")
        print(final_clusters)

        # Record the new predictions
        record_predictions(final_clusters, record_file=record_file)
        print("Model retraining completed successfully.")
    except Exception as e:
        print("Error encountered during retraining:", e)
        # Depending on your needs, you could also log this error to a file or monitoring system.
        raise  # Re-raise the exception so that an external loop/scheduler can decide to retry

# ============
# AUTOMATED RETRAINING & EVALUATION
# ============

def main():
    # Assuming combined_data is a DataFrame that has been loaded previously.
    # For example, it might be loaded from a CSV or database.
    # combined_data = pd.read_csv('your_combined_data.csv', index_col=0)
    #
    # The combined_data is assumed to have a MultiIndex where level 0 is the coin identifier.
    #
    # For demonstration, we'll assume that combined_data is already defined.
    global combined_data  # Ensure that combined_data is defined in your runtime environment

    # First, evaluate older predictions (if any)
    try:
        evaluate_predictions(n_days=3, record_file='predictions_log.csv', updated_data=combined_data)
    except Exception as e:
        print("Error during evaluation:", e)

    # Then, attempt to retrain the model.
    # If an error occurs during retraining, wait and then retry.
    retry_delay_seconds = 60  # Adjust the delay as needed
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            retrain_model(combined_data, record_file='predictions_log.csv')
            break  # Exit loop if retraining was successful
        except Exception as e:
            retries += 1
            print(f"Retraining failed (attempt {retries}/{max_retries}). Retrying in {retry_delay_seconds} seconds...")
            time.sleep(retry_delay_seconds)
    else:
        print("Max retries reached. Please check the error logs and data integrity.")

if __name__ == '__main__':
    main()

No predictions older than 3 days to evaluate.
Starting model retraining...
Retrained Clusters (Predictions):
       mean_price     std_price     min_price     max_price  volatility  \
113  2.824346e+02  4.861554e+01  1.795970e+02  3.815710e+02    0.172130   
148  8.511364e-08  2.359672e-08  5.000000e-08  2.100000e-07    0.277238   
18   1.826719e-02  1.025091e-02  8.491600e-03  3.800430e-02    0.561165   
7    3.192554e-02  8.962780e-03  1.506770e-02  4.961350e-02    0.280740   
156  8.516636e-05  3.057200e-05  4.085000e-05  1.596000e-04    0.358968   
122  4.550650e-02  6.284300e-03  3.922220e-02  5.179080e-02    0.138097   
130  6.181818e-08  7.767276e-09  5.000000e-08  7.000000e-08    0.125647   
55   5.196545e-03  1.497496e-03  2.588360e-03  8.438080e-03    0.288171   
88   6.409087e-03  1.937589e-03  2.750240e-03  1.041550e-02    0.302319   
175  4.228770e-03  1.424566e-03  2.180370e-03  8.279090e-03    0.336875   
19   1.750000e-08  4.826536e-09  1.000000e-08  3.000000e-08    0.2

<ipython-input-8-fadd565f69d6>:189: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_clusters = filtered_features.groupby('Probability_Group', group_keys=False).apply(


In [ ]:
import pandas as pd
import numpy as np
import os
import time  # for sleep between retries
from datetime import datetime, timedelta
from sklearn.cluster import KMeans

# ============
# FUNCTIONS
# ============

def extract_features(data):
    # Ensure date is datetime and sort by date
    prices = data[['date', 'price']].dropna().sort_values('date')
    if len(prices) < 2:
        return None

    prices['date'] = pd.to_datetime(prices['date'])
    first_date = prices['date'].iloc[0]
    last_date = prices['date'].iloc[-1]
    age_in_days = (last_date - first_date).days
    age_in_months = round(age_in_days / 30.44, 2)

    # Function to calculate percentage change over the last 'days' days
    def calculate_price_change(days):
        if len(prices) >= days:
            start_price = prices['price'].iloc[-days]
            end_price = prices['price'].iloc[-1]
            return ((end_price - start_price) / start_price) * 100 if start_price != 0 else 0
        return None

    price_changes = {
        'price_change_24h': calculate_price_change(2),
        'price_change_7d': calculate_price_change(7),
        'price_change_14d': calculate_price_change(14),
        'price_change_30d': calculate_price_change(30)
    }

    # Replace missing changes with default values (here, 0)
    for key in price_changes:
        if price_changes[key] is None:
            price_changes[key] = 0

    # Filter out coins based on conditions
    if (price_changes['price_change_24h'] < 0 and price_changes['price_change_7d'] < 0) or (price_changes['price_change_30d'] < -50):
        return None

    features = {
        'mean_price': np.mean(prices['price']),
        'std_price': np.std(prices['price']),
        'min_price': np.min(prices['price']),
        'max_price': np.max(prices['price']),
        'volatility': np.std(prices['price']) / np.mean(prices['price']) if np.mean(prices['price']) != 0 else 0,
        'price_change': (prices['price'].iloc[-1] - prices['price'].iloc[0]) / prices['price'].iloc[0] if prices['price'].iloc[0] != 0 else 0,
        'token': data['token'].iloc[0] if 'token' in data.columns else None,
        'contract_address': data['contract_address'].iloc[0] if 'contract_address' in data.columns else None,
        'market_cap': data['market_cap'].dropna().iloc[-1] if 'market_cap' in data.columns else None,
        'age_in_months': age_in_months,
        'Chain': data['platform'].iloc[0] if 'platform' in data.columns else None,
        'Trading Volume': data['volume'].dropna().iloc[-1] if 'volume' in data.columns else None,
        'twitter_followers': data['twitter_followers'].iloc[0] if 'twitter_followers' in data.columns else None,
        'price': prices['price'].iloc[-1],
        'prediction_date': data['date'].iloc[-1],
    }

    features.update(price_changes)
    return features

def record_predictions(prediction_df, record_file='predictions_log.csv'):
    """
    Record the prediction data (with a date) to a CSV log.
    The "date" column records the time at which these predictions were made.
    """
    prediction_df = prediction_df.copy()
    # Add the current date of action
    prediction_df['date'] = pd.Timestamp.now()

    # Append to file if exists; otherwise, create new file with header.
    if os.path.exists(record_file):
        prediction_df.to_csv(record_file, mode='a', header=False, index=False)
    else:
        prediction_df.to_csv(record_file, index=False)
    print(f"Predictions recorded to {record_file}")

def evaluate_predictions(n_days=3, record_file='predictions_log.csv', updated_data=None):
    """
    Evaluate predictions made more than n_days ago by comparing the predicted price with the latest actual price.
    The evaluation uses the "date" column from the prediction log.
    `updated_data` is assumed to be the latest combined_data DataFrame with up-to-date prices.
    """
    if not os.path.exists(record_file):
        print("No prediction log file found for evaluation.")
        return

    predictions_log = pd.read_csv(record_file)
    predictions_log['date'] = pd.to_datetime(predictions_log['date'])
    cutoff_date = pd.Timestamp.now() - timedelta(days=n_days)

    # Filter predictions older than n_days
    old_predictions = predictions_log[predictions_log['date'] <= cutoff_date]
    if old_predictions.empty:
        print(f"No predictions older than {n_days} days to evaluate.")
        return

    errors = []
    error_details = []

    # Loop over each old prediction and compute the error between the predicted price and current actual price.
    for idx, row in old_predictions.iterrows():
        coin = row['Coin']
        predicted_price = row['price']
        try:
            # Assumes that `updated_data` has the latest price info, indexed (or grouped) by coin.
            coin_data = updated_data.loc[coin]
            if isinstance(coin_data, pd.DataFrame):
                # If multiple rows exist, use the latest non-NA price
                actual_price = coin_data['price'].dropna().iloc[-1]
            else:
                actual_price = coin_data['price']
        except Exception as e:
            print(f"Could not retrieve data for coin {coin}: {e}")
            continue

        error = actual_price - predicted_price
        errors.append(abs(error))
        error_details.append({
            'Coin': coin,
            'Predicted_Price': predicted_price,
            'Actual_Price': actual_price,
            'Error': error,
            'date': row['date']
        })

    if errors:
        mae = np.mean(errors)
        print(f"Mean Absolute Error over predictions older than {n_days} days: {mae:,.4f}")
        errors_df = pd.DataFrame(error_details)
        print(errors_df)
    else:
        print("No valid predictions were evaluated.")

def retrain_model(combined_data, record_file='predictions_log.csv'):
    """
    Re-compute features, re-cluster coins, and record new predictions.
    The output clustered data includes a "date" column showing the action time.
    Returns the final_clusters DataFrame.
    """
    try:
        print("Starting model retraining...")
        feature_data = []
        # Assuming combined_data is a DataFrame with a MultiIndex where level 0 is the coin identifier.
        for coin, coin_data in combined_data.groupby(level=0):
            features = extract_features(coin_data)
            if features:
                features['Coin'] = coin
                feature_data.append(features)
        if not feature_data:
            print("No valid features extracted for retraining.")
            return None

        features_df = pd.DataFrame(feature_data)

        # Convert 'market_cap' and 'Trading Volume' to numeric (if needed)
        for col in ['market_cap', 'Trading Volume']:
            if col in features_df.columns:
                features_df[col] = pd.to_numeric(features_df[col], errors='coerce')

        # Normalize selected numerical columns for clustering
        norm_columns = ['mean_price', 'std_price', 'volatility', 'price_change']
        features_normalized = features_df[norm_columns].apply(lambda x: (x - x.mean()) / x.std())

        # Apply K-Means clustering
        kmeans = KMeans(n_clusters=3, random_state=42)
        features_df['Cluster'] = kmeans.fit_predict(features_normalized)

        # Map clusters to probability groups based on average price_change
        cluster_mapping = features_df.groupby('Cluster')['price_change'].mean().sort_values(ascending=False).index
        mapping_dict = {}
        if len(cluster_mapping) >= 3:
            mapping_dict = {cluster_mapping[0]: '90% Uptrend',
                            cluster_mapping[1]: '80% Uptrend',
                            cluster_mapping[2]: '70% Uptrend'}
        features_df['Probability_Group'] = features_df['Cluster'].map(mapping_dict)

        # Filter based on trading volume and market cap thresholds
        filtered_features = features_df[
            (features_df['Trading Volume'] >= 50000) & (features_df['market_cap'] >= 1_000_000)
        ]

        # Select exactly 20 coins per probability group (or as many as available if fewer than 20)
        final_clusters = filtered_features.groupby('Probability_Group', group_keys=False).apply(
            lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20)
        )

        # Add the current date to the output clustered data
        final_clusters['date'] = pd.Timestamp.now()

        # Reformat market_cap and Trading Volume for display
        for col in ['market_cap', 'Trading Volume']:
            final_clusters[col] = final_clusters[col].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else x)

        print("Retrained Clusters (Predictions):")
        print(final_clusters)

        # Save/log the prediction results (which now include the action date)
        record_predictions(final_clusters, record_file=record_file)
        print("Model retraining completed successfully.")
        return final_clusters
    except Exception as e:
        print("Error encountered during retraining:", e)
        raise  # Re-raise the exception for external handling

# ============
# AUTOMATED RETRAINING & EVALUATION
# ============

def main():
    global combined_data  # Ensure that combined_data is defined in your runtime environment
    final_clusters = None

    # First, evaluate older predictions (if any)
    try:
        evaluate_predictions(n_days=3, record_file='predictions_log.csv', updated_data=combined_data)
    except Exception as e:
        print("Error during evaluation:", e)

    # Then, attempt to retrain the model.
    # If an error occurs during retraining, wait and then retry.
    retry_delay_seconds = 60  # Adjust the delay as needed
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            final_clusters = retrain_model(combined_data, record_file='predictions_log.csv')
            break  # Exit loop if retraining was successful
        except Exception as e:
            retries += 1
            print(f"Retraining failed (attempt {retries}/{max_retries}). Retrying in {retry_delay_seconds} seconds...")
            time.sleep(retry_delay_seconds)
    else:
        print("Max retries reached. Please check the error logs and data integrity.")

    return final_clusters

if __name__ == '__main__':
    # Assuming combined_data is already loaded in the runtime environment.
    # For example, it might be loaded earlier in the script.
    final_clusters = main()

    # Save clusters to CSV and download if available
    if final_clusters is not None:
        final_clusters.to_csv("final_clustered_coins.csv", index=False)
        from google.colab import files
        files.download("final_clustered_coins.csv")
    else:
        print("No final clusters to save.")


No predictions older than 3 days to evaluate.
Starting model retraining...
Retrained Clusters (Predictions):
       mean_price     std_price     min_price     max_price  volatility  \
149  2.116517e-03  4.223115e-03  6.226500e-04  3.620720e-02    1.995314   
212  2.847178e+02  4.678652e+01  1.795970e+02  3.815710e+02    0.164326   
36   1.002360e-02  5.694250e-03  3.656800e-03  2.171350e-02    0.568084   
10   8.115676e-03  5.037342e-03  2.469670e-03  2.416400e-02    0.620693   
273  6.042128e+00  1.321774e+00  3.906770e+00  8.265430e+00    0.218760   
370  4.274300e-03  1.408981e-03  2.180370e-03  8.279090e-03    0.329640   
420  5.740746e-03  3.092163e-03  1.663280e-03  1.539880e-02    0.538634   
356  1.200200e-02  7.131638e-03  4.637980e-03  3.881600e-02    0.594204   
338  3.409779e-01  1.485477e-01  1.121980e-01  6.753030e-01    0.435652   
270  1.811321e-08  1.099856e-08  1.000000e-08  5.000000e-08    0.607212   
14   3.182203e-02  9.040196e-03  1.506770e-02  4.961350e-02    0.2

<ipython-input-7-d501f7c29e6f>:191: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_clusters = filtered_features.groupby('Probability_Group', group_keys=False).apply(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save clusters to CSV
final_clusters.to_csv("final_clustered_coins.csv", index=False)

# Download the file (if using Colab)
from google.colab import files
files.download("final_clustered_coins.csv")

NameError: name 'final_clusters' is not defined

In [ ]:
#@title Online Learning
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

# =============================================================================
# STEP 1: Feature and Target Extraction (Rolling Window)
# =============================================================================
def generate_training_samples(coin_data, window_size=30, target_offset=7):
    """
    Generate training samples from a coin's historical data using a sliding window.

    Each training sample consists of:
      - Features computed over a window of length `window_size`.
      - A target: the percentage change from the last price in the window to the
        price after an additional `target_offset` days.

    Advanced features include:
      - Basic statistical measures (mean, std, min, max, volatility)
      - Recent price change (over the window and over the last 7 days)
      - Latest market cap and trading volume (if available)
      - Age (in months) of the data window

    Parameters:
      coin_data (DataFrame): Data for one coin; must include at least 'date' and 'price'.
                             Optionally, it can include 'market_cap' and 'volume'.
      window_size (int): Number of rows (days) used for computing features.
      target_offset (int): Number of rows (days) ahead after the window to compute the target.

    Returns:
      samples (list of tuples): Each tuple is (features, target).
    """
    coin_data = coin_data.sort_values('date').copy()
    coin_data['date'] = pd.to_datetime(coin_data['date'])
    samples = []

    # Ensure there are enough rows to form at least one sample.
    if len(coin_data) < window_size + target_offset:
        return samples

    # Slide over the data
    for i in range(len(coin_data) - window_size - target_offset + 1):
        window = coin_data.iloc[i : i+window_size]
        future_window = coin_data.iloc[i+window_size : i+window_size+target_offset]

        # ----- Basic Features -----
        mean_price = window['price'].mean()
        std_price = window['price'].std()
        min_price = window['price'].min()
        max_price = window['price'].max()
        volatility = std_price / mean_price if mean_price != 0 else 0
        price_change_window = ((window['price'].iloc[-1] - window['price'].iloc[0]) /
                               window['price'].iloc[0] if window['price'].iloc[0] != 0 else 0)
        # Price change over the last 7 days of the window (if available)
        if window_size >= 7:
            price_change_7d = ((window['price'].iloc[-1] - window['price'].iloc[-7]) /
                               window['price'].iloc[-7] if window['price'].iloc[-7] != 0 else 0)
        else:
            price_change_7d = 0

        # ----- Advanced Features -----
        market_cap = window['market_cap'].iloc[-1] if 'market_cap' in window.columns else 0
        trading_volume = window['volume'].iloc[-1] if 'volume' in window.columns else 0
        age_in_months = (window['date'].iloc[-1] - window['date'].iloc[0]).days / 30.44

        # ----- Target Computation -----
        current_price = window['price'].iloc[-1]
        future_price = future_window['price'].iloc[-1]
        target = ((future_price - current_price) / current_price if current_price != 0 else 0)

        # Create the feature vector (advanced features appended)
        features = [mean_price, std_price, min_price, max_price, volatility,
                    price_change_window, price_change_7d, market_cap, trading_volume, age_in_months]
        samples.append((features, target))

    return samples

# =============================================================================
# STEP 2: Initialize the Online Learning Model
# =============================================================================

# We use SGDRegressor for online (incremental) learning.
sgd_regressor = SGDRegressor(random_state=42, max_iter=1000, tol=1e-3)
scaler = StandardScaler()  # Scaling helps SGD converge faster

# -----------------------------------------------------------------------------
# Assume `combined_data` is a DataFrame containing historical data for many coins.
# It must include at least the following columns:
#   'date', 'price', and a coin identifier (here, we assume it is 'token').
# Optionally, it may include 'market_cap' and 'volume'.
#
# For example, combined_data might look like:
#    token    date         price    market_cap    volume   ...
#    BTC      2025-01-01   30000    600000000     1000000
#    BTC      2025-01-02   30500    605000000     1100000
#    ...
# -----------------------------------------------------------------------------

# Build the initial training set from historical data:
X_train = []
y_train = []
for coin, coin_data in combined_data.groupby('token'):
    samples = generate_training_samples(coin_data, window_size=30, target_offset=7)
    for features, target in samples:
        X_train.append(features)
        y_train.append(target)

if len(X_train) > 0:
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    # First, fit the scaler on the historical data
    X_train_scaled = scaler.fit_transform(X_train)
    # Train the initial model using partial_fit
    sgd_regressor.partial_fit(X_train_scaled, y_train)
    print("Initial online model trained on historical data with {} samples.".format(len(X_train)))
else:
    print("Not enough historical data for training.")

# =============================================================================
# STEP 3: Online Updating and Prediction Functions
# =============================================================================

def online_update(new_combined_data, window_size=30, target_offset=7):
    """
    Update the online model with new coin data.

    Parameters:
      new_combined_data (DataFrame): Updated data containing new rows for coins.
      window_size (int): Number of days used to compute features.
      target_offset (int): Days ahead to compute the target.
    """
    X_new = []
    y_new = []
    for coin, coin_data in new_combined_data.groupby('token'):
        samples = generate_training_samples(coin_data, window_size, target_offset)
        for features, target in samples:
            X_new.append(features)
            y_new.append(target)
    if len(X_new) > 0:
        X_new = np.array(X_new)
        y_new = np.array(y_new)
        # Scale the new data with the existing scaler
        X_new_scaled = scaler.transform(X_new)
        # Update the model with new samples
        sgd_regressor.partial_fit(X_new_scaled, y_new)
        print("Online model updated with {} new samples.".format(len(X_new)))
    else:
        print("No new training samples available for online update.")

def predict_future_price_change(coin_data, window_size=30):
    """
    Predict the future price change for a given coin using its latest data.

    Parameters:
      coin_data (DataFrame): Historical data for a coin.
      window_size (int): Number of recent days to use for computing features.

    Returns:
      predicted_change (float): The predicted percentage change over the next period.
                                Returns None if there isn’t enough data.
    """
    coin_data = coin_data.sort_values('date').copy()
    coin_data['date'] = pd.to_datetime(coin_data['date'])
    if len(coin_data) < window_size:
        return None
    window = coin_data.iloc[-window_size:]
    # ----- Compute Basic Features -----
    mean_price = window['price'].mean()
    std_price = window['price'].std()
    min_price = window['price'].min()
    max_price = window['price'].max()
    volatility = std_price / mean_price if mean_price != 0 else 0
    price_change_window = ((window['price'].iloc[-1] - window['price'].iloc[0]) /
                           window['price'].iloc[0] if window['price'].iloc[0] != 0 else 0)
    if window_size >= 7:
        price_change_7d = ((window['price'].iloc[-1] - window['price'].iloc[-7]) /
                           window['price'].iloc[-7] if window['price'].iloc[-7] != 0 else 0)
    else:
        price_change_7d = 0

    # ----- Compute Advanced Features -----
    market_cap = window['market_cap'].iloc[-1] if 'market_cap' in window.columns else 0
    trading_volume = window['volume'].iloc[-1] if 'volume' in window.columns else 0
    age_in_months = (window['date'].iloc[-1] - window['date'].iloc[0]).days / 30.44

    features = [mean_price, std_price, min_price, max_price, volatility,
                price_change_window, price_change_7d, market_cap, trading_volume, age_in_months]
    features = np.array(features).reshape(1, -1)
    features_scaled = scaler.transform(features)
    predicted_change = sgd_regressor.predict(features_scaled)[0]
    return predicted_change

# =============================================================================
# STEP 4: Example Usage
# =============================================================================

# Example 1: Predict future price change for a specific coin (e.g., token 'ABC')
coin_ABC_data = combined_data[combined_data['token'] == 'ABC']
predicted_change = predict_future_price_change(coin_ABC_data, window_size=30)
if predicted_change is not None:
    print("Predicted future price change for coin ABC: {:.4f}".format(predicted_change))
else:
    print("Not enough data to predict future price change for coin ABC.")

# Example 2: Perform an online update when new data arrives.
# In practice, new_combined_data could be obtained via an API or a scheduled batch update.
# For example:
# new_combined_data = pd.read_csv('new_data.csv', parse_dates=['date'])
# online_update(new_combined_data, window_size=30, target_offset=7)

# =============================================================================
# Advanced Suggestions for Further Improvement:
# =============================================================================
# 1. **Feature Engineering:**
#    Incorporate additional features such as technical indicators (RSI, MACD, etc.)
#    or sentiment analysis scores from social media.
#
# 2. **Hyperparameter Tuning:**
#    Experiment with different values for window_size, target_offset, and SGDRegressor parameters.
#
# 3. **Model Ensembling:**
#    Consider combining several online models or using ensembles to improve robustness.
#
# 4. **Data Persistence:**
#    Save the scaler and model periodically so that they can be reloaded, avoiding
#    the need to retrain from scratch.
#
# 5. **Error Monitoring:**
#    Implement monitoring and alerting based on prediction errors (e.g., MAE or RMSE)
#    to detect when market conditions change and the model needs adjustment.

Initial online model trained on historical data with 35974 samples.
Not enough data to predict future price change for coin ABC.


In [ ]:
# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
features_df['Cluster'] = kmeans.fit_predict(features_normalized)

# Map clusters to probability groups
cluster_mapping = features_df.groupby('Cluster')['price_change'].mean().sort_values(ascending=False).index
features_df['Probability_Group'] = features_df['Cluster'].map(
    {cluster_mapping[0]: '90% Uptrend',
     cluster_mapping[1]: '80% Uptrend',
     cluster_mapping[2]: '70% Uptrend'}
)

In [ ]:
# Select exactly 20 coins per probability group
final_clusters = (
    features_df.groupby('Probability_Group', group_keys=False)
    .apply(lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20))
)

# Display final clusters
print(final_clusters)

       mean_price     std_price   min_price     max_price  volatility  \
205  2.041926e+02  5.108223e+01  126.591000  2.949300e+02    0.250167   
340  9.506654e+01  2.560423e+01   70.348000  1.805250e+02    0.269330   
457  3.053147e+02  3.521859e+01  252.984000  3.815710e+02    0.115352   
594  1.042537e+02  6.429993e+01   29.406800  2.571680e+02    0.616764   
778  1.083193e+02  1.455196e+01   85.843700  1.404780e+02    0.134343   
872  8.450500e-05  7.660918e-06    0.000071  9.409000e-05    0.090656   
442  4.006520e-03  2.030358e-03    0.000925  7.846780e-03    0.506763   
345  4.773803e-03  5.828563e-03    0.001180  2.523320e-02    1.220948   
742  1.237629e-02  2.452919e-03    0.007320  1.725350e-02    0.198195   
792  3.517404e-03  1.144676e-03    0.001738  7.928140e-03    0.325432   
844  1.141995e-02  6.518991e-03    0.003968  2.421210e-02    0.570842   
527  7.611368e-03  6.981244e-03    0.001622  2.777320e-02    0.917213   
267  2.894176e-03  5.116686e-03    0.000299  2.3470

<ipython-input-6-109ba7c5fccc>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=20, random_state=42) if len(x) >= 20 else x.head(20))
